In [ ]:
from azure.cosmos import CosmosClient
from dotenv import load_dotenv
import json
from openai import AzureOpenAI
import os

def generate_openai_embeddings(input_text):
    client = AzureOpenAI(
    api_key = os.getenv("AZURE_OPENAI_API_KEY"),  
    api_version = "2025-01-01-preview",
    azure_endpoint =os.getenv("AZURE_OPENAI_ENDPOINT") 
    )

    response = client.embeddings.create(
        input = input_text,
        model= "text-embedding-ada-002"
    )

    return response.data[0].embedding

cosmos_url =  os.getenv("COSMOS_ENDPOINT")
cosmos_key =  os.getenv("COSMOS_KEY")
cosmos_database_name = "linkedin_data"
cosmos_container_name = "linkedin_data_vector"
cosmos_client = CosmosClient(cosmos_url, cosmos_key)t(cosmos_url, cosmos_key)
cosmos_container = cosmos_client.get_database_client(cosmos_database_name).get_container_client(cosmos_container_name)

question = "List all employees working at Accenture."

query_embedding = generate_openai_embeddings(question)

top_n = 5

query = """
    SELECT TOP @top_n c.full_name, c.full_name_url, c.role, c.company, VectorDistance(c.request_vector, @query_embedding) AS SimilarityScore
    FROM c
    """
parameters = [
    {"name": "@top_n", "value": top_n},
    {"name": "@query_embedding", "value": query_embedding},
]

result = list(cosmos_container.query_items(
    query=query,
    parameters=parameters,
    enable_cross_partition_query=True
))

print(json.dumps(result, indent=4))

[
    {
        "full_name": "Mukesh Jha",
        "full_name_url": "https://www.linkedin.com/sales/lead/ACwAABtrK0QBT-guG30iiC9bCp1VfflkjC8Ib6I,NAME_SEARCH,6Pgp?_ntb=2l7hMCeUQFKi%2Br3KmkAhjQ%3D%3D",
        "role": "Chief Executive",
        "company": "Bank of India",
        "SimilarityScore": 0.752331808436094
    },
    {
        "full_name": "Harley Young",
        "full_name_url": "https://www.linkedin.com/sales/lead/ACwAAAAQo7IBrIHS6zwtBTaMy-gujutr_oUZd3Y,NAME_SEARCH,MMr_?_ntb=2l7hMCeUQFKi%2Br3KmkAhjQ%3D%3D",
        "role": "Viewed\nYou\u2019ve already seen Harley Young\u2019s profile before.",
        "company": "Amazon Web Services (AWS)",
        "SimilarityScore": 0.7565930281582602
    },
    {
        "full_name": "Barry Robinson",
        "full_name_url": "https://www.linkedin.com/sales/lead/ACwAAAF9RN8BVXGnbcMcb-S4lkkW6mW9cE0jwKI,NAME_SEARCH,QZ4y?_ntb=2l7hMCeUQFKi%2Br3KmkAhjQ%3D%3D",
        "role": "President and Managing Director, International Operations",
        "